# **Self-Driving Car Game with ROS2 and SuperTuxKart**

Welcome to our cool project! We're going to make a car drive by itself in a game called SuperTuxKart using a robot software called ROS2.

## **What We'll Do**
1. Learn a little about ROS2
2. Set up our computer
3. Start SuperTuxKart with ROS2
4. Make the car see the road
5. Make the car drive itself
6. Watch the car race on its own!

## **1. Learning about ROS2**

ROS2 is a special software that helps robots work. Scientists and engineers use it to make robots do things like see, hear, move, and understand the world around them.

## **2. Setting Up Our Computer**

We need to install some software first:

1. **Install ROS2**: Follow the instructions [here](https://docs.ros.org/en/humble/Installation.html).
2. **Install SuperTuxKart**: Download and install it from [here](https://sourceforge.net/projects/maskor/files/stk/ros-humble-supertuxkart.deb).

After installing, open a terminal and type this to start the game with ROS2:

```bash
ros2 run supertuxkart supertuxkart
```

## **3. Starting SuperTuxKart with ROS2**

Let's check if everything is working. Open a terminal and type:

```bash
ros2 node list
```

You should see something like `/stk_node`. To learn more about it, type:

```bash
ros2 node info /stk_node
```

This will show you what the game is sharing with ROS2.

## **4. Making the Car See the Road**

We'll use something called an HSV filter to help the car see the road in the game. Let's see how it works.

### **Code Explanation**

This code gets the game image and helps the car see the road.

In [ ]:
import rclpy
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge
import numpy as np

def nothing(x):
    pass

rclpy.init()
node = rclpy.create_node("get_box_color")
initial_flag = True

def hsv_calc(msg):
    global initial_flag
    if initial_flag:
        initial_flag = False
        cv2.namedWindow("Trackbars")
        cv2.createTrackbar("lh", "Trackbars", 0, 179, nothing)
        cv2.createTrackbar("ls", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("lv", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("uh", "Trackbars", 179, 179, nothing)
        cv2.createTrackbar("us", "Trackbars", 255, 255, nothing)
        cv2.createTrackbar("uv", "Trackbars", 255, 255, nothing)

    frame = CvBridge().imgmsg_to_cv2(msg, "bgr8")
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lh = cv2.getTrackbarPos("lh", "Trackbars")
    ls = cv2.getTrackbarPos("ls", "Trackbars")
    lv = cv2.getTrackbarPos("lv", "Trackbars")
    uh = cv2.getTrackbarPos("uh", "Trackbars")
    us = cv2.getTrackbarPos("us", "Trackbars")
    uv = cv2.getTrackbarPos("uv", "Trackbars")

    l_blue = np.array([lh, ls, lv])
    u_blue = np.array([uh, us, uv])
    mask = cv2.inRange(hsv, l_blue, u_blue)
    result = cv2.bitwise_or(frame, frame, mask=mask)

    cv2.imshow("mask", mask)
    cv2.imshow("result", result)
    key = cv2.waitKey(1)
    if key == 27:
        cv2.destroyAllWindows()
        rclpy.shutdown()

node.create_subscription(Image, "/stk_image", hsv_calc, 1)
rclpy.spin(node)
rclpy.shutdown()

The code does this:

1. Loads important libraries for the project.
2. Creates a function `nothing` for making sliders.
3. Starts ROS2 and creates a node called `get_box_color`.
4. Sets up sliders to adjust colors the first time the function runs.
5. Gets the game image and changes it to a color format called HSV.
6. Uses the slider values to create a mask that finds the road color.
7. Shows the mask and the road found by the mask.
8. Listens to the game images and runs the function for each image.

## **5. Making the Car Drive Itself**

Now, we'll use the HSV filter to make the car follow the road by itself.

### **Code Explanation**

This code helps the car see the road and then control the car to stay on the road.

In [ ]:
import rclpy
from rclpy.node import Node

from sensor_msgs.msg import Image
import cv_bridge
import cv2
from geometry_msgs.msg import Twist
import numpy as np

CONTROL_COEFFICIENT = 0.02


class LaneFollower(Node):
    def __init__(self):
        super().__init__("my_running_node")
        self.create_subscription(
            Image, "/stk_image", self.image_cb, 1
        )
        self.cmd_pub = self.create_publisher(Twist, "/cmd_vel", 1)

    def image_cb(self, msg):
        img = cv_bridge.CvBridge().imgmsg_to_cv2(msg, desired_encoding="bgr8")

        twist_msg = Twist()
        twist_msg.linear.x = 0.3

        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img = cv2.GaussianBlur(img, (5, 5), 0)
        mask = cv2.inRange(img, np.array([92, 0, 90]), np.array([119, 42, 154]))

        # Find the largest segmented contour
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            largest_contour_center = cv2.moments(largest_contour)

            if largest_contour_center["m00"] != 0:
                center_x = int(
                    largest_contour_center["m10"] / largest_contour_center["m00"]
                )
                # Find error (the lane distance from the target distance)
                error = center_x - img.shape[1] / 2
                twist_msg.angular.z = -error * CONTROL_COEFFICIENT
        else:
            twist_msg.linear.x = 0.0
            twist_msg.angular.z = 0.0

        self.cmd_pub.publish(twist_msg)


def main():
    rclpy.init()
    rclpy.spin(LaneFollower())
    rclpy.shutdown()


if __name__ == "__main__":
    main()

The code does this:

1. Loads important libraries for the project.
2. Creates a function `nothing` for making sliders.
3. Starts ROS2 and creates a node called `autonomous_drive`.
4. Creates a way to send drive commands to the car.
5. Sets up sliders to adjust colors the first time the function runs.
6. Gets the game image and changes it to a color format called HSV.
7. Uses the slider values to create a mask that finds the road color.
8. Shows the mask and the road found by the mask.
9. Finds the center of the road.
10. Calculates how far the center of the road is from the center of the image.
11. Makes a command to steer the car towards the road.
12. Sends the command to the car.
13. Listens to the game images and runs the function for each image.

## **6. Running the Final Autonomous Race**

Now that everything is set up, let's run the final autonomous race! Start SuperTuxKart and watch your car drive itself around the track.